In [1]:
#pip install psycopg2

In [2]:
# загрузка библиотек
import psycopg2
import pandas as pd
import getpass

## 1. Загрузка данных

**Значения атрибутов в Sample - Superstore.xls**

| Название столбца | Значение                          |
|------------------|-----------------------------------|
| Row ID           | Идентификатор строки (уникальный) |
| Order ID         | Идентификатор заказа              |
| Order Date       | Дата заказа                       |
| Ship Date        | Дата доставки                     |
| Ship Mode        | Класс доставки                    |
| Customer ID      | Идентификатор покупателя          |
| Customer Name    | Имя и фамилия покупателя          |
| Segment          | Сегмент покупателя                |
| Country          | Страна                            |
| City             | Город                             |
| State            | Штат                              |
| Postal Code      | Почтовый индекс                   |
| Region           | Регион                            |
| Product ID       | Идентификатор товара              |
| Category         | Категория                         |
| Sub-Category     | Подкатегория                      |
| Product Name     | Название товара                   |
| Sales            | Продажи (Доход)                   |
| Quantity         | Количество                        |
| Discount         | Скидка в %                        |
| Profit           | Прибыль                           |
| Person           | Региональный менеджер             |
| Returned         | Возвраты товара                   |

### 1.1 Продажи - Orders

In [3]:
# загрузка данных по продажам
df_orders = pd.read_excel('data/Sample - Superstore.xls', sheet_name='Orders') #index_col='Row ID'
df_orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,2018-06-12,2018-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
# просмотр структуры данных по продажам
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9983 non-null   float64       
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [5]:
# проверка на наличие пропусков
df_orders.isna().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64

Имеется **11** пропусков в данных почтового индекса, атрибут - **'Postal Code'**

In [6]:
# вывод строк с пропущенными значениями почтового индекса
df_orders[df_orders['Postal Code'].isna()][['Country', 'City', 'State', 'Postal Code']]

,Country,City,State,Postal Code
2234,United States,Burlington,Vermont,NaN
5274,United States,Burlington,Vermont,NaN
8798,United States,Burlington,Vermont,NaN
9146,United States,Burlington,Vermont,NaN
9147,United States,Burlington,Vermont,NaN
9148,United States,Burlington,Vermont,NaN
9386,United States,Burlington,Vermont,NaN
9387,United States,Burlington,Vermont,NaN
9388,United States,Burlington,Vermont,NaN
9389,United States,Burlington,Vermont,NaN


Значение почтового индекса отсутствует в городе **Burlington** штат **Vermont**

In [7]:
# проверка наличия значений почтового индекса в остальных данных для города Burlington штат Vermont
df_orders.query('(City == "Burlington") & (State == "Vermont")')[['Country', 'City', 'State', 'Postal Code']]

,Country,City,State,Postal Code
2234,United States,Burlington,Vermont,NaN
5274,United States,Burlington,Vermont,NaN
8798,United States,Burlington,Vermont,NaN
9146,United States,Burlington,Vermont,NaN
9147,United States,Burlington,Vermont,NaN
9148,United States,Burlington,Vermont,NaN
9386,United States,Burlington,Vermont,NaN
9387,United States,Burlington,Vermont,NaN
9388,United States,Burlington,Vermont,NaN
9389,United States,Burlington,Vermont,NaN


Значения почтового индекса для города Burlington штат Vermont **не найдены**

Согласно данным сайта [ZIP Code Query](https://vt.postcodebase.com/city/BURLINGTON) почтовый индекс города ***Burlington*** штат ***Vermont*** **5401**

In [8]:
# заполнение пропусков в данных почтового индекса
df_orders['Postal Code'] = df_orders['Postal Code'].fillna(5401)

In [9]:
# проверка на наличие пропусков
df_orders.isna().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

Пропусков в данных **нет**

In [10]:
# проверка на наличие дубликатов
df_orders.duplicated().sum()

0

In [11]:
# проверка на наличие дубликатов
df_orders.iloc[:,1:].duplicated().sum()

1

В данных по продажам присутствует **1 (один)** дубликат

In [12]:
# вывод дубликатов
df_orders[df_orders.iloc[:,1:].duplicated(keep=False)]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
3405,3406,US-2016-150119,2016-04-23,2016-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,...,43229.0,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
3406,3407,US-2016-150119,2016-04-23,2016-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,...,43229.0,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588


In [13]:
# размерность данных 
df_orders.shape

(9994, 21)

In [14]:
# удаление дубликатов
df_orders = df_orders.drop(index=3405).reset_index(drop=True)
df_orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2018-138688,2018-06-12,2018-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [15]:
# размерность данных 
df_orders.shape

(9993, 21)

Дубликаты **удалены**

### 1.2 Менеджеры - People

In [16]:
# загрузка данных по менеджерам
df_people = pd.read_excel('data/Sample - Superstore.xls', sheet_name='People')
df_people.head()

,Person,Region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


### 1.3 Возвраты - Returns

In [17]:
# загрузка данных по возвратам
df_returns = pd.read_excel('data/Sample - Superstore.xls', sheet_name='Returns')
df_returns.head()

,Returned,Order ID
0,Yes,CA-2016-100762
1,Yes,CA-2016-100762
2,Yes,CA-2016-100762
3,Yes,CA-2016-100762
4,Yes,CA-2016-100867


In [18]:
# проверка на наличие пропусков
df_returns.isna().sum()

Returned    0
Order ID    0
dtype: int64

In [19]:
# проверка на наличие дубликатов
df_returns.duplicated().sum()

504

В данных по возвратам **504** повтора

In [20]:
# размерность данных
df_returns.shape

(800, 2)

In [21]:
# удаление дубликатов
df_returns = df_returns.drop_duplicates().reset_index(drop=True)
df_returns.head()

,Returned,Order ID
0,Yes,CA-2016-100762
1,Yes,CA-2016-100867
2,Yes,CA-2016-102652
3,Yes,CA-2016-103373
4,Yes,CA-2016-103744


In [22]:
# размерность данных
df_returns.shape

(296, 2)

## 2. Подключение к базе данных

In [23]:
# безопасно запрашиваем ввод пароля в CLI
password = getpass.getpass(prompt="Enter secret password:")

Enter secret password:········


In [24]:
# создание подключения к базе данных
connect = psycopg2.connect(database='postgres',
                           user='postgres',
                           password=password,
                           host='127.0.0.1',
                           port='5432')

print('База данных успешно открыта')

База данных успешно открыта


In [25]:
# создание объекта курсора подключения к базе данных 'postgres'
cursor = connect.cursor()

In [26]:
# вывод наименований таблиц в базе данных 'postgres'
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema NOT IN ('information_schema', 'pg_catalog') \
             AND table_schema IN('public', 'myschema');")
cursor.fetchall()

[('orders',), ('people',), ('returns',)]

## 3. Запись данных

### 3.1. Таблица заказы - 'orders'

In [27]:
# значение атрибутов в таблице 'orders'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'orders';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['row_id', 'order_date', 'ship_date', 'postal_code', 'sales', 'quantity', 'discount', 'profit', 'country', 'city', 'state', 'product_name', 'region', 'product_id', 'order_id', 'category', 'subcategory', 'ship_mode', 'customer_id', 'customer_name', 'segment']


In [28]:
# значение атрибутов в исходном файле 'Sample - Superstore.xls'
df_orders.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [29]:
# размерность данных
df_orders.shape

(9993, 21)

In [30]:
# первая строка данных для записи в таблицу 'orders'
order_1 = df_orders.iloc[0].values
order_1

array([1, 'CA-2018-152156', Timestamp('2018-11-08 00:00:00'),
       Timestamp('2018-11-11 00:00:00'), 'Second Class', 'CG-12520',
       'Claire Gute', 'Consumer', 'United States', 'Henderson',
       'Kentucky', 42420.0, 'South', 'FUR-BO-10001798', 'Furniture',
       'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, 2, 0.0,
       41.9136], dtype=object)

In [31]:
# преобразование типа данных для записи в таблицу 'orders'
order_1[[0, 18]] = order_1[[0, 18]].astype('int')

In [32]:
# преобразование типа данных для записи в таблицу 'orders'
order_1[[11, 17, 19, 20]] = order_1[[11, 17, 19, 20]].astype('float')

In [33]:
# первая строка данных для записи в таблицу 'orders'
order_1

array([1, 'CA-2018-152156', Timestamp('2018-11-08 00:00:00'),
       Timestamp('2018-11-11 00:00:00'), 'Second Class', 'CG-12520',
       'Claire Gute', 'Consumer', 'United States', 'Henderson',
       'Kentucky', 42420.0, 'South', 'FUR-BO-10001798', 'Furniture',
       'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, 2, 0.0,
       41.9136], dtype=object)

In [34]:
# задание формата данных даты
cursor.execute("SET datestyle to 'ISO, MDY';")
connect.commit()

In [35]:
# запись первой строки в таблицу - 'orders' 
cursor.execute("INSERT INTO orders( \
                Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, \
                Customer_ID, Customer_Name, Segment, \
                Country, City, State, Postal_Code, Region, \
                Product_ID, Category, SubCategory, Product_Name, Sales, Quantity, Discount, Profit) \
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", order_1)
connect.commit() 

In [36]:
# проверка записи данных в таблицу 'orders'
cursor.execute("SELECT * FROM orders LIMIT 10;")
rows = cursor.fetchall()
cursor.execute("SELECT column_name, ordinal_position \
                FROM information_schema.columns \
                WHERE table_name = 'orders' \
                ORDER BY ordinal_position;")
columnsname = [_[0] for _ in cursor.fetchall()]
df = pd.DataFrame(rows, columns=columnsname)
df

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit
0,1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136000000000000


In [37]:
# данные для записи в таблицу 'orders'
orders = df_orders.iloc[1:].values
orders[:10]

array([[2, 'CA-2018-152156', Timestamp('2018-11-08 00:00:00'),
        Timestamp('2018-11-11 00:00:00'), 'Second Class', 'CG-12520',
        'Claire Gute', 'Consumer', 'United States', 'Henderson',
        'Kentucky', 42420.0, 'South', 'FUR-CH-10000454', 'Furniture',
        'Chairs',
        'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back',
        731.9399999999999, 3, 0.0, 219.58199999999997],
       [3, 'CA-2018-138688', Timestamp('2018-06-12 00:00:00'),
        Timestamp('2018-06-16 00:00:00'), 'Second Class', 'DV-13045',
        'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles',
        'California', 90036.0, 'West', 'OFF-LA-10000240',
        'Office Supplies', 'Labels',
        'Self-Adhesive Address Labels for Typewriters by Universal',
        14.62, 2, 0.0, 6.8713999999999995],
       [4, 'US-2017-108966', Timestamp('2017-10-11 00:00:00'),
        Timestamp('2017-10-18 00:00:00'), 'Standard Class', 'SO-20335',
        "Sean O'Donnell", 'Consumer', '

In [38]:
# преобразование типа данных для записи в таблицу 'orders'
orders[:,[0, 18]] = orders[:,[0, 18]].astype('int')

In [39]:
# преобразование типа данных для записи в таблицу 'orders'
orders[:,[11, 17, 19, 20]] = orders[:,[11, 17, 19, 20]].astype('float')

In [44]:
# данные для записи в таблицу 'orders'
orders[:10]

array([[2, 'CA-2018-152156', Timestamp('2018-11-08 00:00:00'),
        Timestamp('2018-11-11 00:00:00'), 'Second Class', 'CG-12520',
        'Claire Gute', 'Consumer', 'United States', 'Henderson',
        'Kentucky', 42420.0, 'South', 'FUR-CH-10000454', 'Furniture',
        'Chairs',
        'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back',
        731.9399999999999, 3, 0.0, 219.58199999999997],
       [3, 'CA-2018-138688', Timestamp('2018-06-12 00:00:00'),
        Timestamp('2018-06-16 00:00:00'), 'Second Class', 'DV-13045',
        'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles',
        'California', 90036.0, 'West', 'OFF-LA-10000240',
        'Office Supplies', 'Labels',
        'Self-Adhesive Address Labels for Typewriters by Universal',
        14.62, 2, 0.0, 6.8713999999999995],
       [4, 'US-2017-108966', Timestamp('2017-10-11 00:00:00'),
        Timestamp('2017-10-18 00:00:00'), 'Standard Class', 'SO-20335',
        "Sean O'Donnell", 'Consumer', '

In [45]:
# запись данных в таблицу заказы - 'orders'
cursor.executemany("INSERT INTO orders( \
                    Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, \
                    Customer_ID, Customer_Name, Segment, \
                    Country, City, State, Postal_Code, Region, \
                    Product_ID, Category, SubCategory, Product_Name, Sales, Quantity, Discount, Profit) \
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, \
                            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", orders)
connect.commit() 

In [46]:
# проверка записи данных в таблицу 'orders'
cursor.execute("SELECT * FROM orders LIMIT 10;")
rows = cursor.fetchall()
cursor.execute("SELECT column_name, ordinal_position \
                FROM information_schema.columns \
                WHERE table_name = 'orders' \
                ORDER BY ordinal_position;")
columnsname = [_[0] for _ in cursor.fetchall()]
df = pd.DataFrame(rows, columns=columnsname)
df

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,subcategory,product_name,sales,quantity,discount,profit
0,1,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136000000000000
1,2,CA-2018-152156,2018-11-08,2018-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5819999999999700
2,3,CA-2018-138688,2018-06-12,2018-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8713999999999995
3,4,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310000000000600
4,5,US-2017-108966,2017-10-11,2017-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5163999999999990
5,6,CA-2016-115812,2016-06-09,2016-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1693999999999960
6,7,CA-2016-115812,2016-06-09,2016-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656000000000002
7,8,CA-2016-115812,2016-06-09,2016-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152000000000400
8,9,CA-2016-115812,2016-06-09,2016-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825000000000000
9,10,CA-2016-115812,2016-06-09,2016-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4699999999999900


In [47]:
# проверка записи данных в таблицу 'orders'
cursor.execute("SELECT count(*) FROM orders;")
print(cursor.fetchall(), df_orders.shape)

[(9993,)] (9993, 21)


Данные в таблицу **'orders'** записаны **успешно**

### 3.2. Таблица заказы - 'people'

In [48]:
# значение атрибутов в таблице 'people'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'people';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['person', 'region']


In [49]:
# значение атрибутов в исходном файле 'Sample - Superstore.xls'
df_people.columns

Index(['Person', 'Region'], dtype='object')

In [50]:
# данные для записи в таблицу 'people'
people = df_people.values
people

array([['Anna Andreadi', 'West'],
       ['Chuck Magee', 'East'],
       ['Kelly Williams', 'Central'],
       ['Cassandra Brandow', 'South']], dtype=object)

In [51]:
# запись данных в таблицу заказы - 'people' 
cursor.executemany("INSERT INTO people( \
                    Person, Region) \
                    VALUES (%s, %s);", people)
connect.commit() 

In [52]:
# проверка записи данных в таблицу 'people'
cursor.execute("SELECT * FROM people LIMIT 10;")
rows = cursor.fetchall()
cursor.execute("SELECT column_name, ordinal_position \
                FROM information_schema.columns \
                WHERE table_name = 'people' \
                ORDER BY ordinal_position;")
columnsname = [_[0] for _ in cursor.fetchall()]
df = pd.DataFrame(rows, columns=columnsname)
df

,person,region
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


Данные в таблицу **'people'** записаны **успешно**

### 3.3. Таблица заказы - 'returns'

In [53]:
# значение атрибутов в таблице 'returns'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'returns';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['returned', 'order_id']


In [54]:
# значение атрибутов в исходном файле 'Sample - Superstore.xls'
df_returns.columns

Index(['Returned', 'Order ID'], dtype='object')

In [55]:
# данные для записи в таблицу 'returns'
returns = df_returns.values
returns[:10]

array([['Yes', 'CA-2016-100762'],
       ['Yes', 'CA-2016-100867'],
       ['Yes', 'CA-2016-102652'],
       ['Yes', 'CA-2016-103373'],
       ['Yes', 'CA-2016-103744'],
       ['Yes', 'CA-2016-103940'],
       ['Yes', 'CA-2016-104829'],
       ['Yes', 'CA-2016-105270'],
       ['Yes', 'CA-2016-108609'],
       ['Yes', 'CA-2016-108861']], dtype=object)

In [56]:
# запись данных в таблицу заказы - 'returns' 
cursor.executemany("INSERT INTO returns( \
                    Returned, Order_ID) \
                    VALUES (%s, %s);", returns)
connect.commit() 

In [57]:
# проверка записи данных в таблицу 'returns'
cursor.execute("SELECT * FROM returns LIMIT 10;")
rows = cursor.fetchall()
cursor.execute("SELECT column_name, ordinal_position \
                FROM information_schema.columns \
                WHERE table_name = 'returns' \
                ORDER BY ordinal_position;")
columnsname = [_[0] for _ in cursor.fetchall()]
df = pd.DataFrame(rows, columns=columnsname)
df

,returned,order_id
0,Yes,CA-2016-100762
1,Yes,CA-2016-100867
2,Yes,CA-2016-102652
3,Yes,CA-2016-103373
4,Yes,CA-2016-103744
5,Yes,CA-2016-103940
6,Yes,CA-2016-104829
7,Yes,CA-2016-105270
8,Yes,CA-2016-108609
9,Yes,CA-2016-108861


In [58]:
# проверка записи данных в таблицу 'returns'
cursor.execute("SELECT count(*) FROM returns;")
print(cursor.fetchall(), df_returns.shape)

[(296,)] (296, 2)


Данные в таблицу **'returns'** записаны **успешно**

In [59]:
# закрытие подключения к базе данных 'postgres'
connect.close()